In [55]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
sns.set_style('whitegrid')

In [56]:
df_test_features = pd.read_csv('../test_features.csv')

In [57]:
num_test_data = df_test_features.iloc[:,9:]
num_test_data['loudness'] = df_test_features['loudness']
num_test_data['tempo'] = df_test_features['tempo']
num_test_data['duration'] = df_test_features['duration']
cat_test_data = df_test_features[['time_signature', 'key', 'mode']]

In [58]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [59]:
cat_test_data['time_signature_cat'] = le.fit_transform(cat_test_data['time_signature'])
cat_test_data['key_cat'] = le.fit_transform(cat_test_data['key'])
cat_test_data['mode_cat'] = le.fit_transform(cat_test_data['mode'])

C:\Users\alex0\AppData\Local\Temp/ipykernel_29408/719706249.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_test_data['time_signature_cat'] = le.fit_transform(cat_test_data['time_signature'])
C:\Users\alex0\AppData\Local\Temp/ipykernel_29408/719706249.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_test_data['key_cat'] = le.fit_transform(cat_test_data['key'])
C:\Users\alex0\AppData\Local\Temp/ipykernel_29408/719706249.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [60]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [61]:
num_test = scaler.fit_transform(num_test_data)

In [62]:
import ktrain

In [63]:
predictor = ktrain.load_predictor('../text_only_nbsvm')

# Pre-process string with ktrain

In [64]:
def str_remove_comma(string):
    str = string.split(", ")
    return str

def list_to_string(list):
    str = " "
    return str.join(list)

In [65]:
# remove stopwords
import nltk
stopwords = nltk.corpus.stopwords.words('english')

In [66]:
x_test2 = df_test_features['tags'].apply(lambda x : list_to_string(str_remove_comma(x)))
x_test2 = x_test2.apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
x_test2 = np.array(x_test2)

In [67]:
x_text_input = predictor.preproc.preprocess_test(x_test2)

428 test sequences
test sequence lengths:
	mean : 54
	95percentile : 109
	99percentile : 150
x_test shape: (428,150)


# Make a prediction

In [68]:
test_dataset = [x_text_input[0]]

In [69]:
prediction = predictor.model.predict(test_dataset)

In [70]:
predict_labels = [ label.argmax() for label in prediction]
df = pd.DataFrame(predict_labels, columns=['genre_pred'])
classes_mapping = {
    0: 'classic pop and rock',
    1: 'dance and electronica',
    2: 'folk',
    3: 'jazz and blues',
    4: 'metal',
    5: 'pop',
    6: 'punk',
    7: 'soul and reggae',
}
df['genre_pred'] = df['genre_pred'].map(classes_mapping)
df['trackID'] = df_test_features['trackID']

In [71]:
correct_label = pd.read_csv('../test_labels.csv')
df_merged = pd.merge(df, correct_label, on='trackID')

In [72]:
from sklearn.metrics import classification_report
print(classification_report(df_merged['genre'], df_merged['genre_pred']))

                       precision    recall  f1-score   support

 classic pop and rock       0.40      0.09      0.15        64
dance and electronica       0.14      0.02      0.03        64
                 folk       0.44      0.64      0.53        73
       jazz and blues       0.21      0.19      0.20        36
                metal       0.57      0.60      0.58        42
                  pop       0.90      1.00      0.95        44
                 punk       1.00      0.30      0.46        40
      soul and reggae       0.30      0.75      0.43        65

             accuracy                           0.45       428
            macro avg       0.50      0.45      0.42       428
         weighted avg       0.46      0.45      0.40       428

